3. Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:
x - последовательность цифр
y1 = x1 
yi = xi + x1
Если yi >= 10 то yi = yi - 10
Научить модель рекуррентной нейронной сети предсказывать yi по xi Использовать: RNN, LSTM, GRU


In [2]:
import numpy as np
import torch
from itertools import permutations
import random
import time
import torch.nn.functional as F

Подготовка данных
Функция для перестановки заданного списка
Генерируем последовательность из цифр от 0-9. Пример:

[1, 2, 3, 4, 6, 8, 0, 7, 9, 5]
[1, 2, 3, 4, 6, 8, 0, 9, 5, 7]
[1, 2, 3, 4, 6, 8, 0, 9, 7, 5]
[1, 2, 3, 4, 6, 9, 5, 7, 8, 0]
[1, 2, 3, 4, 6, 9, 5, 7, 0, 8]
[1, 2, 3, 4, 6, 9, 5, 8, 7, 0]
[1, 2, 3, 4, 6, 9, 5, 8, 0, 7]

In [5]:
# р закоментировано, так как размер файла становится слишком больши
def permutation(lst):
 
    # Если lst пуст, то перестановок нет
    if len(lst) == 0:
        return []
 
    # если 1 элемент, то одна перестановка возможна
   
    if len(lst) == 1:
        return [lst]
 
    # Найдите перестановки для lst, если они существую
    # более чем 1 символ
 
    l = [] # пустой список, в котором будет храниться текущая перестановка
 
    # Повторите ввод (lst) и вычислите перестановку
    for i in range(len(lst)):
       m = lst[i]
 
       # Extract lst[i] or m from the list.  remLst is
       # remaining list
       remLst = lst[:i] + lst[i+1:]
 
       # Генерация всех перестановок, где m является первым элементом
       
       for p in permutation(remLst):
           l.append([m] + p)
    return l
 
 
# тест функции, добавим все перестановки в список
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
list = []
for p in permutation(data):
    list.append(p)
    #print (p)

In [6]:
list[:5]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [1, 2, 3, 4, 5, 6, 7, 8, 0, 9],
 [1, 2, 3, 4, 5, 6, 7, 9, 8, 0],
 [1, 2, 3, 4, 5, 6, 7, 9, 0, 8],
 [1, 2, 3, 4, 5, 6, 7, 0, 8, 9]]

In [7]:
#длина полученного списка
len(list)

3628800

In [8]:
#отбираем из списка по индексу кратный 50, так как list слишком большой
Xdata = []
for m in range(0, len(list)):
    if m %50==0:
        Xdata.append(list[m])

In [9]:
len(Xdata)

72576

In [10]:
#функция - прописываем условия для Y
def create_dataset(df):
    set = []
    ydata = []
    set.append(df[0])
    for i in range(1, len(df)):
        set.append(df[i]+df[0])
    for j in range(0, len(set)):
        if set[j] >= 10:
            ydata.append(set[j]-10)
        else:
            ydata.append(set[j]) 
    return ydata

In [11]:
#тест
df = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
create_dataset(df)

[1, 3, 4, 5, 6, 7, 8, 9, 0, 1]

In [12]:
#применяем фугкцию для всех элементов списка
Ydata=[]
for k in Xdata:
    Ydata.append(create_dataset(k))

In [13]:
len(Ydata)

72576

In [14]:
Ydata[:5]

[[1, 3, 4, 5, 6, 7, 8, 9, 0, 1],
 [1, 3, 4, 5, 6, 9, 7, 0, 8, 1],
 [1, 3, 4, 5, 6, 1, 7, 0, 8, 9],
 [1, 3, 4, 5, 7, 8, 9, 6, 0, 1],
 [1, 3, 4, 5, 7, 0, 8, 9, 6, 1]]

In [15]:
# делим на train/test
num_trn = int(0.75*len(Xdata))                
X_trn, Y_trn = Xdata[:num_trn], Ydata[:num_trn]
X_test, Y_test = Xdata[num_trn:], Ydata[num_trn:]

In [16]:
len(X_trn)

54432

In [17]:
len(Y_trn)

54432

In [18]:
len(X_test)

18144

In [19]:
len(Y_test)

18144

LSTM

In [21]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # скрытое состояние
        self.hidden_dim = hidden_dim
        # число скрытых слоев
        self.layer_dim = layer_dim
        # LSTM
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True) # batch_first=True (batch_dim, seq_dim, feature_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # инициализация скрытого состояни, 0
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        # инициализация состояни ячейки памяти, 0
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        # Index hidden state of last time step
        out = self.fc(out[:, -1, :]) 
        return out
    
input_dim = 10
hidden_dim = 100
layer_dim = 10
output_dim = 10
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [22]:
model = LSTMModel(input_dim=10, hidden_dim=100, layer_dim=1, output_dim=1)
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Обучаем модель

In [24]:
for epoch in range(50):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
    Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)
    #print(X_trn.shape)
    #print(Y_trn.shape)
    optimizer.zero_grad() 
    out = model.forward(X_trn) 
    #print(out.shape)
    out = out.view(len(Y_trn), -1)
    #print(out.shape)
    #print(Y_trn.shape)
    l = loss(out, Y_trn)
    train_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {train_loss:.4f}",end=" | ")

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([54432, 10])) that is different to the input size (torch.Size([54432, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 28.6768 | 

C:\Users\user\AppData\Local\Temp\ipykernel_7416\2169003629.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_7416\2169003629.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)


Epoch: 1, loss: 24.0056 | Epoch: 2, loss: 19.7648 | Epoch: 3, loss: 15.9122 | Epoch: 4, loss: 12.6498 | Epoch: 5, loss: 10.1838 | Epoch: 6, loss: 8.6565 | Epoch: 7, loss: 8.0824 | Epoch: 8, loss: 8.2937 | Epoch: 9, loss: 8.9006 | Epoch: 10, loss: 9.4534 | Epoch: 11, loss: 9.7238 | Epoch: 12, loss: 9.6987 | Epoch: 13, loss: 9.4611 | Epoch: 14, loss: 9.1149 | Epoch: 15, loss: 8.7513 | Epoch: 16, loss: 8.4376 | Epoch: 17, loss: 8.2144 | Epoch: 18, loss: 8.0958 | Epoch: 19, loss: 8.0730 | Epoch: 20, loss: 8.1212 | Epoch: 21, loss: 8.2081 | Epoch: 22, loss: 8.3021 | Epoch: 23, loss: 8.3780 | Epoch: 24, loss: 8.4202 | Epoch: 25, loss: 8.4234 | Epoch: 26, loss: 8.3908 | Epoch: 27, loss: 8.3323 | Epoch: 28, loss: 8.2608 | Epoch: 29, loss: 8.1897 | Epoch: 30, loss: 8.1303 | Epoch: 31, loss: 8.0902 | Epoch: 32, loss: 8.0722 | Epoch: 33, loss: 8.0746 | Epoch: 34, loss: 8.0918 | Epoch: 35, loss: 8.1163 | Epoch: 36, loss: 8.1403 | Epoch: 37, loss: 8.1572 | Epoch: 38, loss: 8.1636 | Epoch: 39, loss:

Проверка на тестовый данных

In [26]:
for epoch in range(50):
    test_loss = .0
    start_epoch_time = time.time()
    model.eval()
    X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
    Y_test = torch.tensor(Y_test, dtype = torch.float32).view(len(Y_test), -1)
    out = model.forward(X_test) 
    out = out.view(len(Y_test), -1)
    print(out.shape)
    l = loss(out, Y_test)
    test_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {test_loss:.4f}",end=" | ")

torch.Size([18144, 1])
Epoch: 0, loss: 8.7995 | torch.Size([18144, 1])


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([18144, 10])) that is different to the input size (torch.Size([18144, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\user\AppData\Local\Temp\ipykernel_7416\1396939477.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_7416\1396939477.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = torch.tenso

Epoch: 1, loss: 8.7957 | torch.Size([18144, 1])
Epoch: 2, loss: 8.7816 | torch.Size([18144, 1])
Epoch: 3, loss: 8.7650 | torch.Size([18144, 1])
Epoch: 4, loss: 8.7647 | torch.Size([18144, 1])
Epoch: 5, loss: 8.8057 | torch.Size([18144, 1])
Epoch: 6, loss: 8.9043 | torch.Size([18144, 1])
Epoch: 7, loss: 9.0479 | torch.Size([18144, 1])
Epoch: 8, loss: 9.1796 | torch.Size([18144, 1])
Epoch: 9, loss: 9.2129 | torch.Size([18144, 1])
Epoch: 10, loss: 9.1002 | torch.Size([18144, 1])
Epoch: 11, loss: 8.9000 | torch.Size([18144, 1])
Epoch: 12, loss: 8.7645 | torch.Size([18144, 1])
Epoch: 13, loss: 8.8684 | torch.Size([18144, 1])
Epoch: 14, loss: 9.3446 | torch.Size([18144, 1])
Epoch: 15, loss: 10.2329 | torch.Size([18144, 1])
Epoch: 16, loss: 11.4203 | torch.Size([18144, 1])
Epoch: 17, loss: 12.5708 | torch.Size([18144, 1])
Epoch: 18, loss: 13.1889 | torch.Size([18144, 1])
Epoch: 19, loss: 12.9964 | torch.Size([18144, 1])
Epoch: 20, loss: 12.1504 | torch.Size([18144, 1])
Epoch: 21, loss: 11.003

Результаты неудовлетворительные по всем моделям (RNN/LSTM/GRU). Возможно некорректные данные для обучения или ошибки в модели